# Mappings around CIS Controls
The [CIS Controls](https://www.cisecurity.org/controls_pre) are a set of best practices for defending against cyber attacks. The controls are practical and are [mapped](https://www.cisecurity.org/controls/cis-controls-navigator/) to and from popular standards like ISO 27001, NIST SP-800 53 and NIST CSF.

In this notebook these mappings are leveraged to provide a basic mapping from MITRE ATT&CK towards these other frameworks, like ISO 27001. At the time of writing, there exists no publicly available mapping from MITRE ATT&CK to ISO 27001, hence this approach. While it is not the optimal approach, and a dedicated mapping would be more accurate, this method is a starting point.

In [1]:
import pandas as pd

def get_cis_attck_mapping():
    cis_attck_mapping = pd.read_excel("CIS_Controls_v8_ATTCKv82_condensed.xlsx")
    cis_attck_mapping = cis_attck_mapping.drop(["CIS Controls v8","CIS Controls","Asset Type"],axis=1)
    cis_attck_mapping['Technique ID'] = cis_attck_mapping['Technique ID'].str.split(', ')
    return cis_attck_mapping.explode('Technique ID')

In [2]:
ttp_control_map = get_cis_attck_mapping()
print("Number of mappings:",len(ttp_control_map.index))
print("Number of TTPs:",len(ttp_control_map['Technique ID'].value_counts().index))

ttp_control_map

Number of mappings: 2962
Number of TTPs: 383


,Control ID,Control Name,Description,Technique ID
0,1.1,Establish and Maintain Detailed Enterprise Ass...,"Establish and maintain an accurate, detailed, ...",T1200
1,1.2,Address Unauthorized Assets,Ensure that a process exists to address unauth...,T1200
2,1.4,Use Dynamic Host Configuration Protocol (DHCP)...,Use DHCP logging on all DHCP servers or Intern...,T1200
3,2.1,Establish and Maintain a Software Inventory,Establish and maintain a detailed inventory of...,T1560
3,2.1,Establish and Maintain a Software Inventory,Establish and maintain a detailed inventory of...,T1560.001
...,...,...,...,...
104,18.5,Perform Periodic Internal Penetration Tests,Perform periodic internal penetration tests ba...,T1562.006
104,18.5,Perform Periodic Internal Penetration Tests,Perform periodic internal penetration tests ba...,T1137.002
104,18.5,Perform Periodic Internal Penetration Tests,Perform periodic internal penetration tests ba...,T1539
104,18.5,Perform Periodic Internal Penetration Tests,Perform periodic internal penetration tests ba...,T1535


In [3]:
print(ttp_control_map['Technique ID'].value_counts())
ttp_control_map[['Control ID','Control Name']].value_counts()

T1072        29
T1552        28
T1210        28
T1190        24
T1021.001    24
             ..
T1555.001     1
T1498         1
T1203         1
T1218.011     1
T1037.001     1
Name: Technique ID, Length: 383, dtype: int64


Control ID  Control Name                                                                               
4.1         Establish and Maintain a Secure Configuration Process                                          210
18.3        Remediate Penetration Test Findings                                                            169
6.2         Establish an Access Revoking Process                                                           158
6.1         Establish an Access Granting Process                                                           158
18.5        Perform Periodic Internal Penetration Tests                                                    153
                                                                                                          ... 
12.6        Use of Secure Network Management and Communication Protocols                                     1
1.2         Address Unauthorized Assets                                                                      1
4.9     

#### CIS to ISO 27k mapping
Merge the CIS-ATTCK mapping and the CIS-ISO27k mapping and include the control names. Write the result to [an excel file](./ISO_27001_2022_ATTCKv82.xlsx).

In [4]:
def get_iso_attck():
    cis_iso_mapping = pd.read_excel("CIS_Controls_v8_ISO27k_condensed.xlsx")
    cis_iso_mapping = cis_iso_mapping.drop(["CIS Controls v8","CIS Controls","Asset Type"],axis=1)
    cis_iso_mapping['ISO Security Controls'] = cis_iso_mapping['ISO Security Controls'].apply(str).str.split(', ')

    cis_attck_mapping = pd.read_excel("CIS_Controls_v8_ATTCKv82_condensed.xlsx")
    cis_attck_mapping = cis_attck_mapping.drop(["CIS Controls v8","CIS Controls","Asset Type"],axis=1)
    cis_attck_mapping['Technique ID'] = cis_attck_mapping['Technique ID'].str.split(', ')

    cis_attck_iso = cis_attck_mapping.merge(cis_iso_mapping,on="Control ID")
    cis_attck_iso = cis_attck_iso.explode("ISO Security Controls")
    iso_attck = cis_attck_iso[['ISO Security Controls','Technique ID']]
    iso_attck = iso_attck.set_axis(['Control ID','Technique ID'],axis=1)
    iso_attck = iso_attck.explode('Technique ID').drop_duplicates(subset=["Control ID","Technique ID"])
    iso_attck.set_index('Control ID',inplace=True)
    return iso_attck


iso_attck = get_iso_attck()
iso_controls = pd.read_excel("ISO_27001_2022_controls.xlsx", dtype=str)
iso_controls
iso_attck = iso_attck.merge(iso_controls,left_index=True,right_on="Control ID")
iso_attck.set_index('Control ID',inplace=True)

iso_attck.to_excel("ISO_27001_2022_ATTCKv82.xlsx")
iso_attck

,Technique ID,Control Name
Control ID,,
5.9,T1200,Inventory of information and other associated ...
5.9,T1560,Inventory of information and other associated ...
5.9,T1560.001,Inventory of information and other associated ...
5.9,T1059.006,Inventory of information and other associated ...
5.9,T1021.005,Inventory of information and other associated ...
...,...,...
8.29,T1574.009,Security testing in development and acceptance
8.29,T1574.007,Security testing in development and acceptance
8.29,T1574.008,Security testing in development and acceptance
